In [3]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_google_vertexai import ChatVertexAI
from langchain_anthropic import ChatAnthropic
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from typing import Sequence, List
from langgraph.graph import MessageGraph, END
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
import json
import nest_asyncio
nest_asyncio.apply()


#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "Context_files"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    print()
    return "\n".join(lines)

#create inputs to the model, telling it what needs to be done. 
#provides system message
system1 = context_gen("system1.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

refsystem = context_gen("ref_system copy.txt")

system3 = context_gen("system3.txt")

system2 = context_gen("system2.txt")

rag_prompt = context_gen("rag_prompt.txt")


#First input to the model, breaks query down and provides geometry definition

class analyzed_query(BaseModel):
    """Identifying whether a geometric feature is present in the database and stating the name of the feature"""
    answer: str= Field(description="YES or NO")
    features: List= Field(description="Name of the identified geometrical feature (Singular)")

class Response(BaseModel):
    """Listing the answer of whether the feature is present in the database and the name of the feature"""
    Response: List[analyzed_query]
    
#chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
#chat = ChatVertexAI(model_name="gemini-1.0-pro", temperature=0.0, convert_system_message_to_human=True)
chat = ChatAnthropic(model_name = "claude-3-sonnet-20240229", temperature=0.0)
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


In [4]:
'''change the human variable to test different inputs'''
human = ("Select all faces in the xy plane that have a radius greater than 5mm")

base_prompt = ChatPromptTemplate.from_messages([("system", system1), MessagesPlaceholder(variable_name="messages")])
yaml_generator = base_prompt | chat 

#This chain ise used for the reflection prompt, which asks the model to critique and improve on the generated YAML text.
reflection_prompt = ChatPromptTemplate.from_messages([("system", refsystem), few_shot_prompt, MessagesPlaceholder(variable_name="messages")])
reflect = reflection_prompt | chat 
"""
final_output = reflect.invoke({"messages": [HumanMessage(content=str(output))]})
print(final_output.content)
"""
async def generation_node(state: Sequence[BaseMessage]):
        return await yaml_generator.ainvoke({"messages": state}, {"tags": ["loop 004"]})

async def reflection_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
        # Other messages we need to adjust
        cls_map = {"ai": HumanMessage, "human": AIMessage}
        # First message is the original user request. We hold it the same for all nodes
        translated = [state[0]] + [cls_map[msg.type](content=msg.content) for msg in state[1:]]
        res = await reflect.ainvoke({"messages": translated})
        # We treat the output of this as human feedback for the generator
        return HumanMessage(content=res.content)

builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")

def should_continue(state: List[BaseMessage]):
    if len(state) > 3:
        return END
    return "generate"

builder.add_conditional_edges("reflect", should_continue)
builder.add_edge("generate", "reflect")
graph = builder.compile()

async for event in graph.astream([HumanMessage(content=human)]):
        for key, value in event.items():
            if isinstance(value, dict):
                print(value.get('content'))
                print('---')
                print()
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        print(item.get('content'))
                        print('---')
                        print()
                    else:
                        print(item.content)
                        print('---')
                        print()


Select all faces in the xy plane that have a radius greater than 5mm
---

<- filter:>
select * from faces where radius > 5 and normal_zangle between 89 and 91
---

The provided YAML code looks good for selecting all faces in the XY plane that have a radius greater than 5mm:

- filter: select * from faces where radius > 5 and normal_zangle between 89 and 91

A few observations:

1. The query correctly filters the `faces` table.
2. It uses the `radius` column to select faces with radius greater than 5. 
3. The `normal_zangle` condition between 89 and 91 degrees effectively selects faces lying in the XY plane.

This YAML adheres to the specified format of starting with `- filter:` and seems to accurately translate the natural language query. No additional feedback or improvements are needed.
---

Thank you for the feedback. I'm glad the generated YAML code accurately translates the natural language query and adheres to the specified format. I will continue to follow the same approach for 